In [11]:
from twython import TwythonStreamer
import string, json, pprint
import urllib
from datetime import datetime
from datetime import date
from time import *
import string, os, sys, subprocess, time
import psycopg2

In [12]:
APP_KEY = "sVx1KrI5JMAtoxA1ILHKY34y5"
APP_SECRET = "MGRwmjv61TcW3jCjB7oIDR7SBY3wzgNJMr9ots9UWqhUHvRwnc"
OAUTH_TOKEN = "1625512214-Tw55NjJh5XmnsDkCMJTeU7R1R69PUa5IlC09tNw"
OAUTH_TOKEN_SECRET = "hKvtsUv7M5oVhNhbF6vwIEivOmnSL3Xd8gsrcxacFgOcb"

In [38]:
#Class to process JSON data comming from the twitter stream API. Extract relevant fields
output_file = 'result_'+datetime.now().strftime('%Y%m%d-%H%M%S')+'.csv' 
target = open(output_file, 'a')
class MyStreamer(TwythonStreamer):
    def on_success(self, data):
         tweet_lat = 0.0
         tweet_lon = 0.0
         tweet_name = ""
         retweet_count = 0

         if 'id' in data:
               tweet_id = data['id']
         if 'text' in data:
               tweet_text = data['text'].encode('utf-8').replace("'","''").replace(';','')
         if 'coordinates' in data:    
               geo = data['coordinates']
               if not geo is None:
                    latlon = geo['coordinates']
                    tweet_lon = latlon[0]
                    tweet_lat= latlon[1]
         if 'created_at' in data:
                    dt = data['created_at']
                    tweet_datetime = datetime.strptime(dt, '%a %b %d %H:%M:%S +0000 %Y')

         if 'user' in data:
                    users = data['user']
                    tweet_name = users['screen_name']

         if 'retweet_count' in data:
                    retweet_count = data['retweet_count']
                    
         if tweet_lat != 0:
                    #some elementary output to console    
                    string_to_write = str(tweet_datetime)+", "+str(tweet_lat)+", "+str(tweet_lon)+": "+str(tweet_text)
                    print string_to_write
                    #write_tweet(string_to_write)
         target.write(str(tweet_id))
         target.write(',')
         target.write(str(tweet_datetime))
         target.write(',')
         target.write(tweet_name)
         target.write(',')
         target.write(str(retweet_count))
         target.write(',')         
         target.write(str(tweet_lat))
         target.write(',')
         target.write(str(tweet_lon))
         target.write('\n')
               
    def on_error(self, status_code, data):
         print "OOPS FOUTJE: " +str(status_code)
         #self.disconnect

In [39]:
#main procedure
def main():
    try:
        stream = MyStreamer(APP_KEY, APP_SECRET,OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
        print 'Connecting to twitter: will take a minute'
    except ValueError:
        print 'OOPS! that hurts, something went wrong while making connection with Twitter: '+str(ValueError)
    #global target
    
    
    # Filter based on bounding box see twitter api documentation for more info
    try:
        stream.statuses.filter(locations='3.00,50.00,7.35,53.65')
    except ValueError:
        print 'OOPS! that hurts, something went wrong while getting the stream from Twitter: '+str(ValueError)


                
if __name__ == '__main__':
    main()

Connecting to twitter: will take a minute
2016-01-22 09:18:14, 51.2542, 7.1483: Dienstfahrt  ist beim national express 442 367 (mit GREAT BRITAIN Schrift #bahnbilderde https://t.co/iSCzJbiN78 https://t.co/cXcCwQInLQ
2016-01-22 09:18:27, 51.2542, 7.1483: Am IC 1025 von Hamburg-Altona nach Koblenz Hauptbahnhof ist am 20.01.2016  #bahnbilderde https://t.co/RzvNUfpwlc https://t.co/5iVuuaS3bo
2016-01-22 09:18:30, 50.8845847, 5.75383086: Blauw Sjuut meert weer in de gemeente Meerssen https://t.co/UrHGDEzneE (Meerssen.nl)
2016-01-22 09:18:36, 51.2542, 7.1483: RB 48 nach Köln Hbf mit zwei national express Talent 2 (158 + 151) bei ei #bahnbilderde https://t.co/ZFuEkNk6P7 https://t.co/QYHhQ5qpTw
2016-01-22 09:18:39, 50.92727, 4.42575: #BusinessMgmt #Job in #Vilvoorde: Consulting Project Technical Manager at Oracle https://t.co/lhonnJVIPU #Jobs #Hiring #CareerArc
2016-01-22 09:18:40, 51.87946667, 4.53592848: B1 AMBU 17114 Maasstadweg 3079DZ Rotterdam ROTTDM bon 7277 https://t.co/9AoFRyKDhv #p2000

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:120: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


KeyboardInterrupt: 

In [33]:
import psycopg2
import sys
import pprint

try:
    conn = psycopg2.connect("dbname='demo' user='user' host='localhost' password='user'")
except:
    print "I am unable to connect to the database"
cur = conn.cursor()
cur.executemany("""INSERT INTO bar(tweet_id,tweet_lat,tweet_lon) VALUES (%(tweet_id)s, %(tweet_lat)s,%(tweet_lon)s)""",output_file)

 

TypeError: string indices must be integers, not str